<a href="https://colab.research.google.com/github/Pedro1Guevara/ine_2024/blob/main/proyecto_ine_elecciones_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Configuration and Setup
This section handles the initial setup of our dashboard. We install all necessary libraries and import them. The key libraries are:

* dash and dash-bootstrap-components for the web interface
* plotly for interactive visualizations
* pandas for data manipulation
* numpy for numerical operations




These libraries will allow us to create an interactive web-based dashboard with responsive visualizations.

In [39]:
# Install required libraries
!pip install dash dash-bootstrap-components pandas plotly

# Import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import numpy as np

# Part 2: Data Loading and Preprocessing
This section focuses on data preparation. We:

* Define a function to clean monetary values
* Load data from GitHub repositories
* Clean and preprocess both income and expenses datasets
* Handle missing values and string-to-numeric conversions

The clean_money function is particularly important as it handles various edge cases in the monetary data.

In [40]:
def clean_money(x):
    """Cleans monetary values and converts them to float"""
    if pd.isna(x) or x == '' or x == '-':
        return 0
    try:
        cleaned = str(x).replace('$', '').replace(',', '').strip()
        return float(cleaned)
    except ValueError:
        return 0

# Load datasets from GitHub
income_url = "https://raw.githubusercontent.com/Pedro1Guevara/ine_2024/master/17.-ING_RUBRO_CAM_PRELO_23_24_240624.csv"
expenses_url = "https://raw.githubusercontent.com/Pedro1Guevara/ine_2024/master/18.-GAS_RUBRO_CAM_PRELO_23_24_240624.csv"

df_income = pd.read_csv(income_url)
df_expenses = pd.read_csv(expenses_url)

# Define monetary columns for cleaning
monetary_columns_income = [
    'APORTACIONES DE MILITANTES', 'APORTACIONES DE SIMPATIZANTES',
    'APORTACIONES DEL CANDIDATO', 'APORTACIONES DEL CANDIDATO INDEPENDIENTE',
    'FINANCIAMIENTO PÚBLICO', 'OTROS INGRESOS', 'RENDIMIENTOS BANCARIOS',
    'RENDIMIENTOS FINANCIEROS, FONDOS Y FIDEICOMISOS',
    'TRANSFERENCIAS DE CANDIDATOS R.P. FEDERALES',
    'TRANSFERENCIAS DE CANDIDATOS R.P. LOCALES',
    'TRANSFERENCIAS DE CONCENTRADORAS', 'TOTAL INGRESOS'
]

monetary_columns_expenses = [
    'FINANCIEROS', 'OPERATIVOS DE LA CAMPAÑA',
    'PRODUCCIÓN DE LOS MENSAJES PARA RADIO Y T.V.',
    'PROPAGANDA', 'PROPAGANDA EN DIARIOS, REVISTAS Y OTROS MEDIOS IMPRESOS',
    'PROPAGANDA EN VÍA PÚBLICA', 'PROPAGANDA EXHIBIDA EN SALAS DE CINE',
    'PROPAGANDA UTILITARIA',
    'REDES SOCIALES Y PROPAGANDA EXHIBIDA EN PÁGINAS DE INTERNET',
    'TOTAL GASTOS'
]

# Clean monetary columns
for col in monetary_columns_income:
    if col in df_income.columns:
        df_income[col] = df_income[col].apply(clean_money)

for col in monetary_columns_expenses:
    if col in df_expenses.columns:
        df_expenses[col] = df_expenses[col].apply(clean_money)

# Part 3: Dashboard Layout
This section defines the complete visual structure of our dashboard. The layout includes:

* A main title

* Multiple tabs for different analyses:

  * Party Overview: Shows coalition analysis and party-level comparisons
  * Income and Expenses Composition: Displays detailed breakdowns of all financial components
  * State Analysis: Provides state-level financial comparisons
  * Gender Analysis: Shows gender distribution across multiple dimensions


Each tab contains responsive visualizations with consistent styling
Bootstrap grid system for optimal display and mobile responsiveness

In [41]:
# Initialize Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Campaign Finance Dashboard", className="text-center mb-4")
        ])
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Tabs([
                # Tab 1: Party Overview
                dcc.Tab(label='Party Overview', children=[
                    dbc.Row([
                        dbc.Col([
                            dcc.Graph(id='coalition-analysis'),
                        ], width=12),
                        dbc.Col([
                            dcc.Graph(id='party-income-vs-expenses'),
                        ], width=12),
                        dbc.Col([
                            dcc.Graph(id='party-balance'),
                        ], width=12)
                    ])
                ]),

                # Tab 2: Income and Expenses Composition
                dcc.Tab(label='Income and Expenses Composition', children=[
                    dbc.Row([
                        dbc.Col([
                            dcc.Graph(id='income-composition'),
                        ], width=12),
                        dbc.Col([
                            dcc.Graph(id='expenses-composition'),
                        ], width=12)
                    ])
                ]),

                # Tab 3: State Analysis
                dcc.Tab(label='State Analysis', children=[
                    dbc.Row([
                        dbc.Col([
                            dcc.Graph(id='state-income-vs-expenses'),
                        ], width=12),
                        dbc.Col([
                            dcc.Graph(id='state-balance'),
                        ], width=12)
                    ])
                ]),

                # Tab 4: Gender Analysis
                dcc.Tab(label='Gender Analysis', children=[
                    dbc.Row([
                        dbc.Col([
                            dcc.Graph(id='gender-distribution'),
                        ], width=12),
                        dbc.Col([
                            dcc.Graph(id='gender-party-expenses'),
                        ], width=12),
                        dbc.Col([
                            dcc.Graph(id='gender-party-count'),
                        ], width=12),
                        dbc.Col([
                            dcc.Graph(id='gender-state-count'),
                        ], width=12)
                    ])
                ])
            ])
        ])
    ])
], fluid=True)

# Part 4: Interactive Callbacks
This section contains all callback functions for generating interactive visualizations. Key features include:

Coalition Analysis:

* Categorizes parties into main political coalitions
* Calculates and displays total income and expenses by coalition
* Shows percentage distribution of resources
* Includes debug functionality for coalition verification

Income and Expenses Analysis:

* Complete breakdown of all income sources
* Detailed visualization of expense categories
* Party-level comparative analysis
* Balance calculations with color coding for deficits/surpluses


State Analysis:

* Geographic distribution of resources
* State-level income vs expenses comparison
* Balance analysis by state
* Sorting by financial volume


Gender Analysis:

* Overall gender distribution of resources
* Party-level gender expense analysis
* Candidate count by party and gender
* State-level gender distribution
* Comparative analysis of resource allocation



Each callback includes:

* Data preprocessing and cleaning
* Sorting and organization of information
* Consistent formatting of monetary values
* Interactive elements (tooltips, hover information)
* Responsive design elements
* Clear titles and labels

The callbacks are structured to ensure:

* Efficient data processing
* Consistent ordering across visualizations
* Proper alignment of comparative data
* Clear presentation of financial information
* Easy interaction for users

In [42]:
# Helper function for coalition categorization
def categorize_party(party_name):
    if any(name in party_name for name in ['MORENA', 'PT', 'PVEM']):
        return 'Coalición MORENA'
    elif any(name in party_name for name in ['PRI', 'PAN', 'PRD']):
        return 'Coalición PRI-PAN-PRD'
    elif any(name in party_name for name in ['MC', 'MOVIMIENTO C']):
        return 'Movimiento Ciudadano'
    else:
        return 'Otros'

# Debug function for coalition verification
def print_coalition_debug():
    df_income['Coalition'] = df_income['SIGLAS'].apply(categorize_party)

    # Print parties in each coalition
    for coalition in df_income['Coalition'].unique():
        parties = df_income[df_income['Coalition'] == coalition]['SIGLAS'].unique()
        print(f"\nParties in {coalition}:")
        print(parties)

    # Verify sums
    coalition_sums = df_income.groupby('Coalition')['TOTAL INGRESOS'].sum()
    print("\nSums by coalition:")
    print(coalition_sums)

    # Verify total sum matches
    total_sum = df_income['TOTAL INGRESOS'].sum()
    coalition_total = coalition_sums.sum()
    print(f"\nTotal sum: ${total_sum:,.2f}")
    print(f"Coalition sum: ${coalition_total:,.2f}")
    print(f"Difference: ${total_sum - coalition_total:,.2f}")

# Callback: Coalition Analysis
@app.callback(
    Output('coalition-analysis', 'figure'),
    Input('coalition-analysis', 'id')
)
def update_coalition_analysis(dummy):
    df_income_temp = df_income.copy()
    df_expenses_temp = df_expenses.copy()

    # Add coalition categories
    df_income_temp['Coalition'] = df_income_temp['SIGLAS'].apply(categorize_party)
    df_expenses_temp['Coalition'] = df_expenses_temp['SIGLAS'].apply(categorize_party)

    # Calculate totals and percentages
    income_by_coalition = df_income_temp.groupby('Coalition')['TOTAL INGRESOS'].sum().sort_values(ascending=False)
    expenses_by_coalition = df_expenses_temp.groupby('Coalition')['TOTAL GASTOS'].sum()
    expenses_by_coalition = expenses_by_coalition[income_by_coalition.index]

    total_income = income_by_coalition.sum()
    total_expenses = expenses_by_coalition.sum()
    income_pct = income_by_coalition / total_income * 100
    expenses_pct = expenses_by_coalition / total_expenses * 100

    fig = go.Figure()

    # Add income bars
    fig.add_trace(go.Bar(
        name='Income',
        x=income_by_coalition.index,
        y=income_by_coalition.values,
        text=[f'${x:,.0f}<br>({y:.1f}%)' for x, y in zip(income_by_coalition.values, income_pct)],
        textposition='auto',
        marker_color='rgb(55, 83, 109)',
        hovertemplate='Income: $%{y:,.0f}<br>%{text}<extra></extra>'
    ))

    # Add expense bars
    fig.add_trace(go.Bar(
        name='Expenses',
        x=expenses_by_coalition.index,
        y=expenses_by_coalition.values,
        text=[f'${x:,.0f}<br>({y:.1f}%)' for x, y in zip(expenses_by_coalition.values, expenses_pct)],
        textposition='auto',
        marker_color='rgb(26, 118, 255)',
        hovertemplate='Expenses: $%{y:,.0f}<br>%{text}<extra></extra>'
    ))

    fig.update_layout(
        title='Income vs Expenses by Political Coalition',
        barmode='group',
        height=500,
        yaxis_title='Amount (MXN)',
        xaxis_title='Political Coalition',
        showlegend=True,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        )
    )

    return fig

# Callback: Party Income vs Expenses Analysis
@app.callback(
    Output('party-income-vs-expenses', 'figure'),
    Input('party-income-vs-expenses', 'id')
)
def update_party_income_vs_expenses(dummy):
    income_by_party = df_income.groupby('SIGLAS')['TOTAL INGRESOS'].sum().sort_values(ascending=False)
    expenses_by_party = df_expenses.groupby('SIGLAS')['TOTAL GASTOS'].sum()
    expenses_by_party = expenses_by_party[income_by_party.index]

    fig = go.Figure()

    # Add income bars
    fig.add_trace(go.Bar(
        name='Income',
        x=income_by_party.index,
        y=income_by_party.values,
        text=[f'${x:,.0f}' for x in income_by_party.values],
        textposition='auto',
        marker_color='rgb(55, 83, 109)'
    ))

    # Add expense bars
    fig.add_trace(go.Bar(
        name='Expenses',
        x=expenses_by_party.index,
        y=expenses_by_party.values,
        text=[f'${x:,.0f}' for x in expenses_by_party.values],
        textposition='auto',
        marker_color='rgb(26, 118, 255)'
    ))

    fig.update_layout(
        title='Income vs Expenses by Party',
        barmode='group',
        height=500,
        yaxis_title='Amount (MXN)',
        xaxis_title='Political Party'
    )

    return fig

# Callback: Party Balance
@app.callback(
    Output('party-balance', 'figure'),
    Input('party-balance', 'id')
)
def update_party_balance(dummy):
    income_by_party = df_income.groupby('SIGLAS')['TOTAL INGRESOS'].sum().sort_values(ascending=False)
    expenses_by_party = df_expenses.groupby('SIGLAS')['TOTAL GASTOS'].sum()

    balance = pd.DataFrame({
        'Party': income_by_party.index,
        'Balance': income_by_party - expenses_by_party[income_by_party.index]
    })

    colors = ['red' if x < 0 else 'green' for x in balance['Balance']]

    fig = go.Figure(go.Bar(
        x=balance['Party'],
        y=balance['Balance'],
        text=[f'${x:,.0f}' for x in balance['Balance']],
        textposition='auto',
        marker_color=colors
    ))

    fig.update_layout(
        title='Balance (Income - Expenses) by Party',
        height=500,
        yaxis_title='Balance (MXN)',
        xaxis_title='Political Party'
    )

    return fig

# Callback: Complete Income Composition
@app.callback(
    Output('income-composition', 'figure'),
    Input('income-composition', 'id')
)
def update_income_composition(dummy):
    parties = df_income.groupby('SIGLAS')['TOTAL INGRESOS'].sum().sort_values(ascending=False).index
    income_categories = [col for col in monetary_columns_income if col != 'TOTAL INGRESOS']

    fig = go.Figure()
    for category in income_categories:
        values = df_income.groupby('SIGLAS')[category].sum()
        fig.add_trace(
            go.Bar(
                name=category,
                x=parties,
                y=values,
                text=[f'${v:,.0f}' for v in values],
                textposition='auto'
            )
        )

    fig.update_layout(
        title='Complete Income Composition by Party',
        barmode='stack',
        height=600,
        yaxis_title='Amount (MXN)',
        xaxis_title='Political Party',
        showlegend=True
    )

    return fig

# Callback: Complete Expenses Composition
@app.callback(
    Output('expenses-composition', 'figure'),
    Input('expenses-composition', 'id')
)
def update_expenses_composition(dummy):
    parties = df_expenses.groupby('SIGLAS')['TOTAL GASTOS'].sum().sort_values(ascending=False).index
    expense_categories = [col for col in monetary_columns_expenses if col != 'TOTAL GASTOS']

    fig = go.Figure()
    for category in expense_categories:
        values = df_expenses.groupby('SIGLAS')[category].sum()
        fig.add_trace(
            go.Bar(
                name=category,
                x=parties,
                y=values,
                text=[f'${v:,.0f}' for v in values],
                textposition='auto'
            )
        )

    fig.update_layout(
        title='Complete Expenses Composition by Party',
        barmode='stack',
        height=600,
        yaxis_title='Amount (MXN)',
        xaxis_title='Political Party',
        showlegend=True
    )

    return fig

# Callback: State Analysis
@app.callback(
    Output('state-income-vs-expenses', 'figure'),
    Input('state-income-vs-expenses', 'id')
)
def update_state_income_vs_expenses(dummy):
    income_by_state = df_income.groupby('ESTADO ELECCION')['TOTAL INGRESOS'].sum().sort_values(ascending=False)
    expenses_by_state = df_expenses.groupby('ESTADO ELECCION')['TOTAL GASTOS'].sum()
    expenses_by_state = expenses_by_state[income_by_state.index]

    fig = go.Figure()

    fig.add_trace(go.Bar(
        name='Income',
        x=income_by_state.index,
        y=income_by_state.values,
        text=[f'${x:,.0f}' for x in income_by_state.values],
        textposition='auto',
        marker_color='rgb(55, 83, 109)'
    ))

    fig.add_trace(go.Bar(
        name='Expenses',
        x=expenses_by_state.index,
        y=expenses_by_state.values,
        text=[f'${x:,.0f}' for x in expenses_by_state.values],
        textposition='auto',
        marker_color='rgb(26, 118, 255)'
    ))

    fig.update_layout(
        title='Income vs Expenses by State',
        barmode='group',
        height=500,
        yaxis_title='Amount (MXN)',
        xaxis_title='State'
    )
    fig.update_xaxes(tickangle=45)

    return fig

# Callback: State Balance
@app.callback(
    Output('state-balance', 'figure'),
    Input('state-balance', 'id')
)
def update_state_balance(dummy):
    income_by_state = df_income.groupby('ESTADO ELECCION')['TOTAL INGRESOS'].sum().sort_values(ascending=False)
    expenses_by_state = df_expenses.groupby('ESTADO ELECCION')['TOTAL GASTOS'].sum()

    balance = pd.DataFrame({
        'State': income_by_state.index,
        'Balance': income_by_state - expenses_by_state[income_by_state.index]
    })

    colors = ['red' if x < 0 else 'green' for x in balance['Balance']]

    fig = go.Figure(go.Bar(
        x=balance['State'],
        y=balance['Balance'],
        text=[f'${x:,.0f}' for x in balance['Balance']],
        textposition='auto',
        marker_color=colors
    ))

    fig.update_layout(
        title='Balance (Income - Expenses) by State',
        height=500,
        yaxis_title='Balance (MXN)',
        xaxis_title='State'
    )
    fig.update_xaxes(tickangle=45)

    return fig

# Callback: Gender Analysis
@app.callback(
    [Output('gender-distribution', 'figure'),
     Output('gender-party-expenses', 'figure'),
     Output('gender-party-count', 'figure'),
     Output('gender-state-count', 'figure')],
    [Input('gender-distribution', 'id')]
)
def update_gender_analysis(dummy):
    # Overall gender distribution
    gender_income = df_income.groupby('SEXO')['TOTAL INGRESOS'].sum()

    fig1 = go.Figure(go.Pie(
        labels=['Men', 'Women'],
        values=[gender_income['H'], gender_income['M']],
        text=[f'${v:,.0f}' for v in [gender_income['H'], gender_income['M']]],
        hole=0.4
    ))

    fig1.update_layout(
        title='Income Distribution by Gender',
        height=500
    )

    # Expenses by party and gender
    gender_party_expenses = df_expenses.groupby(['SIGLAS', 'SEXO'])['TOTAL GASTOS'].sum().unstack(fill_value=0)
    gender_party_expenses = gender_party_expenses.reindex(
        df_expenses.groupby('SIGLAS')['TOTAL GASTOS'].sum().sort_values(ascending=False).index
    )

    fig2 = go.Figure()
    fig2.add_trace(go.Bar(
        name='Men',
        x=gender_party_expenses.index,
        y=gender_party_expenses['H'],
        text=[f'${x:,.0f}' for x in gender_party_expenses['H']],
        textposition='auto'
    ))

    fig2.add_trace(go.Bar(
        name='Women',
        x=gender_party_expenses.index,
        y=gender_party_expenses['M'],
        text=[f'${x:,.0f}' for x in gender_party_expenses['M']],
        textposition='auto'
    ))

    fig2.update_layout(
        title='Expenses by Party and Gender',
        barmode='group',
        height=500,
        yaxis_title='Amount (MXN)',
        xaxis_title='Political Party'
    )

    # Candidate count by party and gender
    candidate_count = df_income.groupby(['SIGLAS', 'SEXO']).size().unstack(fill_value=0)
    candidate_count = candidate_count.reindex(
        df_income.groupby('SIGLAS')['TOTAL INGRESOS'].sum().sort_values(ascending=False).index
    )

    fig3 = go.Figure()
    fig3.add_trace(go.Bar(
        name='Men',
        x=candidate_count.index,
        y=candidate_count['H'],
        text=candidate_count['H'],
        textposition='auto'
    ))

    fig3.add_trace(go.Bar(
        name='Women',
        x=candidate_count.index,
        y=candidate_count['M'],
        text=candidate_count['M'],
        textposition='auto'
    ))

    fig3.update_layout(
        title='Number of Candidates by Party and Gender',
        barmode='group',
        height=500,
        yaxis_title='Number of Candidates',
        xaxis_title='Political Party'
    )

    # Candidate count by state and gender
    state_gender_count = df_income.groupby(['ESTADO ELECCION', 'SEXO']).size().unstack(fill_value=0)
    state_gender_count = state_gender_count.reindex(
        df_income.groupby('ESTADO ELECCION')['TOTAL INGRESOS'].sum().sort_values(ascending=False).index
    )

    fig4 = go.Figure()
    fig4.add_trace(go.Bar(
        name='Men',
        x=state_gender_count.index,
        y=state_gender_count['H'],
        text=state_gender_count['H'],
        textposition='auto'
    ))

    fig4.add_trace(go.Bar(
        name='Women',
        x=state_gender_count.index,
        y=state_gender_count['M'],
        text=state_gender_count['M'],
        textposition='auto'
    ))

    fig4.update_layout(
        title='Number of Candidates by State and Gender',
        barmode='group',
        height=500,
        yaxis_title='Number of Candidates',
        xaxis_title='State'
    )
    fig4.update_xaxes(tickangle=45)

    return fig1, fig2, fig3, fig4

# Run the dashboard
if __name__ == '__main__':
    # Print debug information about coalitions
    print_coalition_debug()

    try:
        # Try to use Colab-specific display
        from google.colab import output
        output.serve_kernel_port_as_window(8050)
        app.run_server(debug=True, port=8050, mode='external')
    except:
        # Fallback for local execution
        app.run_server(debug=True, port=8050)


Parties in Coalición PRI-PAN-PRD:
['PAN' 'PRI' 'PRD' 'PRI-PAN-PRD' 'PAN-PRI-PRD'
 'PAN-PRI-PRD-PARTIDO REDES SOCIALES PROGRESISTAS MORELOS'
 'PRI-PRD-UNIDAD DEMOCRÁTICA DE COAHUILA' 'PAN-PRI-PRD-PSI' 'PRI-PRD'
 'PAN-PRI' 'PAN-PRI-PRD-PAS' 'MICHOACAN PRIMERO'
 'PARTIDO POPULAR CHIAPANECO' 'PAN-PRI-PRD-NUEVA ALIANZA ESTADO DE MÉXICO']

Parties in Coalición MORENA:
['PT' 'PVEM' 'PVEM-PT-MORENA' 'MORENA' 'MORENA-PT-PVEM'
 'MORENA-PT-PVEM-HAGAMOS-FUTURO'
 'MORENA-PT-PVEM-NUEVA ALIANZA MORELOS-MOVIMIENTO ALTERNATIVA SOCIAL-PARTIDO ENCUENTRO SOCIAL MORELOS'
 'MORENA-PT-PVEM-PARTIDO ENCUENTRO SOLIDARIO CHIAPAS-PARTIDO CHIAPAS UNIDO-PODEMOS MOVER A CHIAPAS-REDES SOCIALES PROGRESISTAS CHIAPAS-PARTIDO POPULAR CHIAPANECO-FUERZA POR MÉXICO CHIAPAS'
 'PVEM-PT-MORENA-FUERZA POR MÉXICO VERACRUZ'
 'PT-NUEVA ALIANZA ZACATECAS-ENCUENTRO SOLIDARIO ZACATECAS'
 'MORENA-PT-PVEM-NUEVA ALIANZA PUEBLA-FUERZA POR MEXICO PUEBLA'
 'PT-MORENA' 'MORENA-PVEM' 'PT-PVEM-MORENA' 'PVEM-MORENA'
 'PVEM-NUEVA ALIANZA PUEBL

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>